In [2]:
import pandas as pd
import pickle
import numpy as np
import torch
from torch import nn
import os 
import fastai
import fastai
from fastai.text import *
PATH="/home/kirana/Documents/phd/final"
DATAPATH="/home/kirana/Documents/phd/data/aclImdb"

## Dataset

In [3]:
[df_train,df_valid,df_test,itos, train_tokens, valid_tokens, test_tokens, trn_lm, val_lm, test_lm]=pickle.load(open(f'{PATH}/inter/dfs_tokens_fastai.pkl','rb'))

In [4]:
type(df_train)

pandas.core.frame.DataFrame

In [5]:
df_train=pd.concat([df_train,df_valid],axis=0)

In [6]:
df_train.shape

(75000, 4)

In [7]:
df_valid=df_test.copy()

In [8]:
df_train=df_train.loc[df_train['label']<2]

In [9]:
df_train.shape

(25000, 4)

In [10]:
df_train.head()

,label,text,words,tokens
3218,0,I've noticed how all the other reviews of this...,"[ \n , xxbos, xxfld, 1, i, 've, noticed, how, ...","[41, 42, 43, 40, 13, 161, 2030, 110, 44, 3, 10..."
6854,0,I went on a visit to one of my relatives a whi...,"[ \n , xxbos, xxfld, 1, i, went, on, a, visit,...","[41, 42, 43, 40, 13, 436, 30, 7, 2074, 9, 39, ..."
2697,0,Darcy and her young daughter Pamela are headin...,"[ \n , xxbos, xxfld, 1, xxmaj, darcy, and, her...","[41, 42, 43, 40, 2, 9225, 6, 55, 207, 551, 2, ..."
5747,0,Battlestar Gallactica was so great because it ...,"[ \n , xxbos, xxfld, 1, xxmaj, battlestar, xxm...","[41, 42, 43, 40, 2, 7900, 2, 33242, 20, 52, 10..."
8731,0,"Any movie with ""National Lampoon"" in the title...","[ \n , xxbos, xxfld, 1, xxmaj, any, movie, wit...","[41, 42, 43, 40, 2, 120, 25, 23, 16, 2, 2155, ..."


In [11]:
type(df_train['tokens'])

pandas.core.series.Series

In [12]:
bs=52 # 52 - Jeremey, 20 - default

In [13]:
bptt= 70 #70 - Jeremey, 35 - default

In [14]:
df_train.head()

,label,text,words,tokens
3218,0,I've noticed how all the other reviews of this...,"[ \n , xxbos, xxfld, 1, i, 've, noticed, how, ...","[41, 42, 43, 40, 13, 161, 2030, 110, 44, 3, 10..."
6854,0,I went on a visit to one of my relatives a whi...,"[ \n , xxbos, xxfld, 1, i, went, on, a, visit,...","[41, 42, 43, 40, 13, 436, 30, 7, 2074, 9, 39, ..."
2697,0,Darcy and her young daughter Pamela are headin...,"[ \n , xxbos, xxfld, 1, xxmaj, darcy, and, her...","[41, 42, 43, 40, 2, 9225, 6, 55, 207, 551, 2, ..."
5747,0,Battlestar Gallactica was so great because it ...,"[ \n , xxbos, xxfld, 1, xxmaj, battlestar, xxm...","[41, 42, 43, 40, 2, 7900, 2, 33242, 20, 52, 10..."
8731,0,"Any movie with ""National Lampoon"" in the title...","[ \n , xxbos, xxfld, 1, xxmaj, any, movie, wit...","[41, 42, 43, 40, 2, 120, 25, 23, 16, 2, 2155, ..."


In [15]:
df_train['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [16]:
df_valid['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [17]:
itos[:5]

['_unk_', '_pad_', 'xxmaj', 'the', '.']

In [18]:
padlen=1400
padding_idx=1

In [19]:
df_train['n_tok']=df_train['tokens'].apply(len)

In [20]:
df_train['n_tok'].describe()

count    25000.000000
mean       304.945840
std        224.981807
min         16.000000
25%        166.000000
50%        228.000000
75%        371.000000
max       3354.000000
Name: n_tok, dtype: float64

In [21]:
df_train.sort_values(by='n_tok', inplace=True)

In [22]:
df_valid['n_tok']=df_valid['tokens'].apply(len)
df_valid.sort_values(by='n_tok', inplace=True)

In [23]:
df_test['n_tok']=df_test['tokens'].apply(len)
df_test.sort_values(by='n_tok', inplace=True)

In [24]:
for i in range(0,20):
    x=df_train['tokens'].values[i]
    y=df_train['label'].values[i]    
    out=np.ones(padlen)
    if len(x) < padlen:
        out[:len(x)]=x
    else:
        out=x[:padlen]
    print (out)
    print (y)
    print (len(out))
    if i>10:
        break

[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
1
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
1
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
1
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400


In [25]:
df_train.shape

(25000, 5)

In [26]:
df_train['label'].value_counts(), df_valid['label'].value_counts(),df_test['label'].value_counts()

(1    12500
 0    12500
 Name: label, dtype: int64, 1    12500
 0    12500
 Name: label, dtype: int64, 1    12500
 0    12500
 Name: label, dtype: int64)

In [27]:
df_train.shape, df_valid.shape

((25000, 5), (25000, 5))

In [28]:
padlen

1400

In [29]:
class ds_sentiment (Dataset):
    def __init__ (self,df,bs,padlen=64,xvar='tokens',yvar='label',len_var='n_tok',padding_idx=1):
        self.x,self.y,self.padlen,self.padding_idx,self.len_var,self.bs=\
            df[xvar],df[yvar],padlen,padding_idx,df[len_var],bs
        self.len_var=self.len_var.clip(0,padlen)
    
    def pad (self,x):
        out=np.ones(self.padlen)*self.padding_idx
        out=out.astype(int)
        if len(x)>=self.padlen:
            out[:]=x[:self.padlen]
        else:
            out[:len(x)]=x
        return out
        
    def __getitem__(self,idx):
        #import pdb
        #pdb.set_trace()
        return self.pad(self.x.iloc[idx]),self.y.iloc[idx],self.len_var.iloc[idx]
    
    def __len__(self):
        return len(self.x)        


In [30]:
#padlen=max(df_train['n_tok'])
padlen=1400

In [31]:
padlen

1400

In [32]:
dstrain=ds_sentiment(df_train,bs,padlen)
dsvalid=ds_sentiment(df_valid,bs,padlen)
dstest=ds_sentiment(df_test,bs,padlen)

In [33]:
dltrain=DataLoader(dstrain,bs,True)
dlvalid=DataLoader(dsvalid,bs,False)
dltest=DataLoader(dstest,bs,False)

In [34]:
for xb,yb,xlen in dltrain:
    break

In [35]:
xb, yb, xlen

(tensor([[41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1],
         ...,
         [41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1]]),
 tensor([0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1,
         0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1,
         1, 0, 0, 1]),
 tensor([163, 195, 186, 483, 780, 721, 131, 193, 186, 198, 146, 178, 578, 144,
         227, 180, 253, 309, 127, 325, 121, 133, 287, 643, 118, 275, 296,  71,
         219, 159, 961, 426, 185, 168, 672, 142, 173, 521,  99, 184, 408,  73,
         499, 438, 214, 390, 179, 644, 178, 333, 529,  75]))

## Model Architecture

In [36]:
from torch import nn
import torch
from torch.autograd import Variable

In [37]:
bs,bptt

(52, 70)

In [69]:
n_inp=len(itos)
n_emb=400 #650
n_hidden=256#400
n_layers= 2 # 2
dropout=0.5 # 0.5
wd=1e-7
bidirectional=True
dropout_e=0.1 # 0.5
dropout_o=0.5 #0.5
n_out=1

In [70]:
xb.shape,xlen.shape

(torch.Size([52, 1400]), torch.Size([52]))

In [71]:
df_train['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [72]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [73]:
itos[:5]

['_unk_', '_pad_', 'xxmaj', 'the', '.']

In [155]:
class sentiment_classifier (nn.Module):
    def __init__(self,n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=0.05,dropout=0.5,\
                 dropout_o=0.5,pretrain_mtx=None,n_out=1,padding_idx=1):
        super().__init__()
        self.n_inp,self.n_emb,self.n_hidden,self.n_layers,self.bidirectional,self.bs,self.device,self.pretrain_mtx,self.padding_idx=\
                            n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,pretrain_mtx,padding_idx
        self.n_out=n_out
        self.dropout_e,self.dropout,self.dropout_o=dropout_e,dropout,dropout_o
        
        self.create_architecture()
        if pretrain_mtx is not None:
            print (f'initializing glove with {pretrain_mtx.shape}')
            self.initialize_glove()
        self.init_hidden()
        self.criterion=nn.BCEWithLogitsLoss()
        
    def create_architecture(self):
        # Dropout layer
        self.dropout_enc=nn.Dropout(self.dropout_e)
        # Embedding Layer: Embedding layer just maps each word to an index. n_inp to n_emb mapping is all it does
            # input to this is of shape n_batch * n_seq
        self.encoder=nn.Embedding(self.n_inp,self.n_emb,padding_idx=self.padding_idx)
             # embs are going to be of shape n_batch * n_seq * n_emb
            # h and c are of shape n_layers * n_batch * n_hidden
        # LSTM Layer
        self.lstm=nn.LSTM(self.n_emb,self.n_hidden,self.n_layers,batch_first=True,dropout=self.dropout,\
                          bidirectional=self.bidirectional)
        
                # output is of shape n_batch * n_seq * n_hidden
            # output [:,-1,:] is the same as hn[-1]
        self.dropout_op=nn.Dropout(self.dropout_o)
        self.fc=nn.Linear(self.n_hidden*2,self.n_out)

        #self.log_softmax=nn.LogSoftmax()
        self.sigmoid=nn.Sigmoid()
    
    def freeze_embedding(self):
        self.encoder.weight.requires_grad=False
         
    def unfreeze_embedding(self):
        self.encoder.weight.requires_grad=True

    def initialize_glove(self):
        self.encoder.weight.data.copy_(torch.Tensor(self.pretrain_mtx))
    
    def init_hidden(self):
        # Initialize hidden
        self.hidden=(Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)),
                     Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)))
    
        
    def forward(self,Xb,Yb,Xb_lengths):
        embs=self.dropout_enc(self.encoder(Xb))
        
        #import pdb
        #pdb.set_trace()
        #embs=self.dropout_enc(embs)
        if Xb.size(0) < self.bs:
            self.hidden=(self.hidden[0][:,:Xb.size(0),:].contiguous(),
            self.hidden[1][:,:Xb.size(0),:].contiguous())

        #import pdb
        #pdb.set_trace()
        # before lstm call
            # pack_padded_sequence so that padded items in the sequence won't be shown to the LSTM
        packed_embs = pack_padded_sequence(embs,Xb_lengths,batch_first=True, enforce_sorted=False)
        lstm_out,(hidden,cell)=self.lstm(packed_embs)

        # after lstm call
        lstm_out,_=pad_packed_sequence(lstm_out,batch_first=True)
        #lstm_out=self.dropout_op(lstm_out)      
        
        # Wrap the hidden state in a new tensor without the gradients
        #self.hidden=(Variable(newhidden[0].data,requires_grad=False).to(self.device),Variable(newhidden[1].data,requires_grad=False).to(self.device))
        
            # lstm_out has the values that are padded
        #out=self.fc (lstm_out[:,-1,:])
        hidden = self.dropout_op(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        
        preds=self.fc(hidden)
        
              
        
        loss=self.criterion(preds,Yb.contiguous().float().view(-1,1))
        
        return preds.view(-1), loss  

In [156]:
def accuracy_binomial(preds,actual, device="cpu", cutoff=0.5):
    preds=torch.sigmoid(preds)
    zeros=torch.zeros(len(preds)).to(device)
    ones = torch.ones(len(preds)).to(device)

    preds=torch.where(preds>cutoff,ones,zeros)
    correct=torch.round(preds).long()==actual
    return correct.float().sum()/len(correct)

In [176]:
def accuracy_binomial(preds, y, device="cpu", cutoff=0.5):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    y=y.float()
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [158]:
device="cpu"

## Load pretrained model weights

In [159]:
pretrained_lm_weights=pickle.load(open(f'{PATH}/inter/pretrained_lm_weights','rb'))

In [160]:
if 1==0:
    model=torch.load (f'{PATH}/inter/model_awd_lstm')
    pretrained_lm_weights=model.encoder.weight.data.cpu().numpy()
    import pickle
    pickle.dump(pretrained_lm_weights,open(f'{PATH}/inter/pretrained_lm_weights','wb'))

### Check if model forward works

In [161]:
n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e,dropout_o,dropout,n_out

(59972, 400, 256, 2, True, 52, 'cpu', 0.1, 0.5, 0.5, 1)

In [162]:
model_sentiment=sentiment_classifier (n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=dropout_e,dropout=dropout,\
                 dropout_o=dropout_o,pretrain_mtx=pretrained_lm_weights,n_out=1,padding_idx=1)


initializing glove with (59972, 400)


In [163]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [164]:
print(f'The model has {count_parameters(model_sentiment):,} trainable parameters')

The model has 26,913,857 trainable parameters


In [165]:
model_sentiment

sentiment_classifier(
  (dropout_enc): Dropout(p=0.1)
  (encoder): Embedding(59972, 400, padding_idx=1)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (dropout_op): Dropout(p=0.5)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (sigmoid): Sigmoid()
  (criterion): BCEWithLogitsLoss()
)

In [166]:
model_sentiment.forward(xb,yb,xlen)

(tensor([ 0.0370, -0.0034, -0.0273,  0.0088, -0.0391, -0.0137,  0.0986,  0.0247,
          0.0145,  0.0296,  0.0379,  0.0119, -0.0146,  0.0033,  0.0184,  0.0458,
          0.0027, -0.0416,  0.0547, -0.0632,  0.0379,  0.0348,  0.0184,  0.0241,
         -0.0462,  0.0375,  0.0077,  0.0640,  0.0176,  0.0270, -0.0055,  0.0371,
          0.0323,  0.0594,  0.0674,  0.0308,  0.0122,  0.0433, -0.0048,  0.0195,
         -0.0300,  0.0382, -0.0103,  0.0123,  0.0502, -0.0274,  0.0332, -0.0077,
          0.0094,  0.0810,  0.0735,  0.0239], grad_fn=<ViewBackward>),
 tensor(0.6939, grad_fn=<BinaryCrossEntropyWithLogitsBackward>))

In [167]:
preds,loss=model_sentiment.forward(xb,yb,xlen)

In [168]:
preds.view(-1)

tensor([ 0.0328,  0.0304, -0.0412, -0.0206, -0.0045,  0.0226, -0.0414,  0.0050,
         0.0140,  0.0043,  0.0076,  0.0084, -0.0293,  0.0179,  0.0486,  0.0119,
        -0.0007,  0.0232, -0.0146,  0.0175, -0.0438, -0.0003,  0.0812,  0.0051,
         0.0064,  0.0305,  0.0179,  0.0673,  0.0232, -0.0281,  0.0026,  0.0331,
        -0.0094,  0.0824, -0.0050,  0.0266,  0.0409,  0.0280, -0.0154,  0.0162,
         0.0267,  0.0305,  0.0503,  0.0686, -0.0723, -0.0213,  0.0341,  0.0272,
         0.0087,  0.0547, -0.0005,  0.0212], grad_fn=<ViewBackward>)

In [169]:
yb

tensor([0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1,
        0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1,
        1, 0, 0, 1])

In [170]:
preds.size()

torch.Size([52])

In [171]:
yb.size()

torch.Size([52])

In [177]:
accuracy_binomial(preds.to(device),yb.to(device))

tensor(0.6154)

In [178]:
from sklearn.metrics import roc_auc_score

In [179]:
roc_auc_score(yb,preds)

0.5684523809523809

In [180]:
pretrained_lm_weights

array([[-9.038397e-02, -1.795423e-02, -3.457930e-02,  8.671192e-02, ..., -3.003510e-02, -1.899482e-02,  1.426260e-02,
        -4.948893e-02],
       [-2.453323e-02, -3.018817e-02,  6.663179e-02, -5.793565e-02, ...,  1.840935e-02,  6.371137e-02,  9.835491e-03,
        -2.131385e-03],
       [ 7.696263e-03, -4.266643e-02, -1.533517e-01,  2.239776e-01, ...,  9.869517e-02,  3.041433e-02,  1.824751e-01,
         1.134978e-01],
       [ 3.305928e-02,  2.266591e-01, -4.264669e-02,  1.490862e-01, ...,  3.407921e-02, -6.422209e-03,  3.180612e-01,
         9.373549e-02],
       ...,
       [ 6.027538e-02, -5.980809e-02,  1.861691e-01, -3.105092e-02, ..., -2.764457e-02,  1.962678e-02, -2.172215e-03,
         6.297247e-02],
       [-1.592789e-02, -2.781571e-04,  1.301994e-01,  2.851282e-02, ...,  5.064877e-02,  1.670864e-01,  2.283701e-02,
        -8.746398e-03],
       [ 4.669700e-02,  3.138980e-02,  1.221957e-02, -3.927753e-02, ..., -1.281436e-01,  1.121320e-01, -3.726090e-03,
        -3.777364e

## Learner

In [192]:
class Learner():
    def __init__(self,model,optimizer,metric_fn,device,bptt=12,print_every=5,clip_val=None):
        self.model,self.optimizer,self.metric_fn,self.device,self.print_every,self.bptt,self.losses,self.clip_val=\
            model,optimizer,metric_fn,device,print_every,bptt,[],clip_val
        self.n_epochs=1
  
        
    
    def fit (self,Xb,Yb,Xlen,mode_train=True):
        if mode_train:
            self.model.train()
        else:
            self.model.eval()
            
        preds,loss=self.model(Xb,Yb,Xlen)
        
        with torch.no_grad():
            acc=self.metric_fn(preds,Yb.view(-1),self.device)
            acc=acc.item()
            del preds
        
        if mode_train:
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        myloss=loss.item()
        del loss
        
        if self.clip_val is not None:
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.clip_val)
        
        return myloss, acc
    
    def lr_find (self,start_lr,end_lr,iterator,n_batch):
        losses,lrs=[],[]
        ratio=end_lr/start_lr
        num_steps=n_batch
        lr=start_lr
        for i in range(num_steps):            
            lr=lr*(end_lr/start_lr)**(1/num_steps)
            lrs.append(lr)
        self.lrs=lrs
        self.run_epoch(iterator,mode_train=True,lrs=lrs)
    
    def run_epoch(self,iterator,mode_train,lrs=None):
        epoch_loss,epoch_acc,i,k=0,0,0,0
        self.model.init_hidden()
        for Xb,Yb,Xlen in iterator:
            Xb=Xb.to(self.device)
            Yb=Yb.to(self.device)
            Xlen=Xlen.to(self.device)
            
            if lrs is not None:
                lr=lrs[k]
                for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr 
            

            loss,acc=self.fit(Xb,Yb,Xlen,mode_train)
            
            if lrs is not None:
                self.losses.append(loss)
            
            
            epoch_loss+=loss
            epoch_acc+=acc
            
            k=k+1
            if k%self.print_every == 0:
                if k:
                    print (f'Batch:{k} {epoch_loss/(k)}  {epoch_acc/(k)}')  
                    torch.cuda.empty_cache()
        epoch_loss=epoch_loss/len(iterator)
        epoch_acc=epoch_acc/len(iterator)
            
        return epoch_loss,epoch_acc
    
    def plot_lrs(self, n_roll=1):
        import seaborn as sns
        ax=sns.lineplot(x=self.lrs,y=pd.Series(self.losses).rolling(n_roll).mean())
        ax.set_xscale('log')
        ax.set_ylabel('Loss')
        ax.set_xlabel('Learning Rate')

    
    def run_epochs(self,dltrain,dlvalid,n_epochs=1):
        for epoch in range(n_epochs):
            
            loss,acc=self.run_epoch(dltrain,True)
            print (f'Epoch:{epoch} Loss:{loss}')
            lossv,accv=self.run_epoch(dlvalid,mode_train=False)
            print (f'Epoch:{epoch} Loss:{loss} Accuracy:{acc} Loss:{lossv} Accuracy:{accv}')
        

In [182]:
device="cuda:1"

In [183]:
n_layers

2

In [184]:
model_sentiment=sentiment_classifier (n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e,dropout,\
                 dropout_o,pretrain_mtx=pretrained_lm_weights,n_out=1,padding_idx=1)
model_sentiment=model_sentiment.to(device)

initializing glove with (59972, 400)


In [185]:
optimizer=torch.optim.Adam(model_sentiment.parameters(),lr=5e-3,betas=(0.9,0.999), weight_decay=wd)
metric_fn=accuracy_binomial

In [186]:
len(dltrain),len(dlvalid)

(481, 481)

In [187]:
df_valid['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [193]:
learner=Learner(model_sentiment,optimizer,accuracy_binomial,device,bptt,50,0.25)

In [189]:
model_sentiment.freeze_embedding()

In [194]:
model_sentiment.encoder.weight.requires_grad

False

In [195]:
learner.run_epochs(dltrain,dlvalid,2)

Batch:50 0.5091784131526947  0.7565384888648987
Batch:100 0.5364243876934052  0.7430769515037536
Batch:150 0.5347875092426936  0.7488461820284525
Batch:200 0.5449489161372185  0.7508654120564461
Batch:250 0.5468099616765976  0.7416154119968414
Batch:300 0.5276621927817663  0.7532692583401998
Batch:350 0.5068982922605105  0.7661538745675768
Batch:400 0.5015247979387641  0.7677404129505158
Batch:450 0.4991620348228349  0.7676496013005575
Epoch:0 Loss:0.4942282327061631
Batch:50 0.355908137857914  0.8634615731239319
Batch:100 0.3485264804959297  0.8705769592523575
Batch:150 0.3436759972572327  0.8719231128692627
Batch:200 0.3474301767349243  0.8717308053374291
Batch:250 0.34870263147354125  0.8725384979248046
Batch:300 0.35452267120281855  0.8685897797346115
Batch:350 0.3576361414364406  0.8669231130395617
Batch:400 0.3612840761989355  0.8632692669332027
Batch:450 0.36427672386169435  0.8609829417864482
Epoch:0 Loss:0.4942282327061631 Accuracy:0.7702463104422523 Loss:0.3675293359216186 Ac

In [196]:
learner.run_epochs(dltrain,dlvalid,3)

Batch:50 0.4389213413000107  0.8034615659713745
Batch:100 0.4099065914750099  0.8228846460580825
Batch:150 0.39467354128758114  0.8310256727536519
Batch:200 0.3872016042470932  0.8354808008670807
Batch:250 0.3768967933654785  0.8412308025360108
Batch:300 0.36916590328017873  0.8450000337759653
Batch:350 0.35638657901968274  0.8507143202849797
Batch:400 0.3503705378249288  0.8541827273368835
Batch:450 0.340133024652799  0.8598290944099426
Epoch:0 Loss:0.3355939168165479
Batch:50 0.27864652171730997  0.8907692682743072
Batch:100 0.2630013384670019  0.895961577296257
Batch:150 0.25939228440324463  0.8966667060057322
Batch:200 0.2598541658371687  0.8972115778923034
Batch:250 0.2614097537994385  0.8966923468112945
Batch:300 0.2638008652627468  0.8946795250972112
Batch:350 0.26257163716214044  0.895494543654578
Batch:400 0.2623019848018885  0.8950961923599243
Batch:450 0.26286310863163737  0.8953419184684753
Epoch:0 Loss:0.3355939168165479 Accuracy:0.861986280974628 Loss:0.26179640820702993 

In [198]:
model_sentiment.unfreeze_embedding()

In [199]:
learner.run_epochs(dltrain,dlvalid,2)

Batch:50 0.18434305623173713  0.9311538863182068
Batch:100 0.1962388039007783  0.9273077327013016
Batch:150 0.1926337090631326  0.9289743983745575
Batch:200 0.20348883811384438  0.9237500393390655
Batch:250 0.2029773797392845  0.9226923470497131
Batch:300 0.2005410532404979  0.9235256789127986
Batch:350 0.20049869877951484  0.9236813567365919
Batch:400 0.19790935169905424  0.9245192690193653
Batch:450 0.19522838408748308  0.9250427736176385
Epoch:0 Loss:0.19435510527257363
Batch:50 0.21288682252168656  0.9196154284477234
Batch:100 0.1971728055551648  0.9242308115959168
Batch:150 0.18927811918159326  0.9265385023752848
Batch:200 0.1866662477143109  0.92740388661623
Batch:250 0.18581973095238208  0.9268461937904358
Batch:300 0.18825404573231935  0.9256410656372706
Batch:350 0.18948654982660498  0.9253846553393773
Batch:400 0.18937665530480444  0.9250481162965297
Batch:450 0.18985555245644517  0.924487218591902
Epoch:0 Loss:0.19435510527257363 Accuracy:0.9253998466935822 Loss:0.1902294925

In [179]:
len(dltrain)

433

In [185]:
model_sentiment.unfreeze_embedding()

In [186]:
learner.run_epochs(dltrain,dlvalid,1)

Batch:50 0.6453262722492218  0.6419231033325196
Batch:100 0.646092454791069  0.6440384846925735
Batch:150 0.6475411713123321  0.6400000242392222
Batch:200 0.6472525778412819  0.6382692554593086
Batch:250 0.6482151463031769  0.6372307926416397
Batch:300 0.648103004693985  0.6371154076854388
Batch:350 0.6477661645412445  0.637967055950846
Batch:400 0.647689622193575  0.6379807917028666
Epoch:0 Loss:0.6474101744402914
Epoch:0 Loss:0.6474101744402914 Accuracy:0.6383487004743713 Loss:0.6549209563099608 Accuracy:0.6357928058322595


In [187]:
learner.run_epochs(dltrain,dlvalid,3)

Batch:50 0.6463013350963592  0.6476923274993897
Batch:100 0.6445563387870789  0.6455769440531731
Batch:150 0.6444319272041321  0.6460256640116374
Batch:200 0.6456667852401733  0.642500022649765
Batch:250 0.6460841224193573  0.6393846383094788
Batch:300 0.6460355154673258  0.6408333570758502
Batch:350 0.6468912330695561  0.6384066168751036
Batch:400 0.6467960438132286  0.6388461776077747
Epoch:0 Loss:0.6463942212540889
Epoch:0 Loss:0.6463942212540889 Accuracy:0.6399475645798733 Loss:0.6536961599272124 Accuracy:0.6306907619748797
Batch:50 0.6493575870990753  0.6342307943105697
Batch:100 0.647639335989952  0.6423077157139778
Batch:150 0.6473417774836222  0.6419231015443801
Batch:200 0.6454289036989213  0.6459615634381771
Batch:250 0.6455069897174835  0.6428461775779725
Batch:300 0.6450929262240728  0.6438461778561274
Batch:350 0.6450396980558123  0.6428571672098977
Batch:400 0.6445977027714253  0.6433173318207264
Epoch:1 Loss:0.6451607778771645
Epoch:1 Loss:0.6451607778771645 Accuracy:0.6

In [86]:
learner.device=device
learner.model=learner.model.to(device)
model.device=device
learner.model.device=device
learner.model.hidden=(learner.model.hidden[0].cuda(1),learner.model.hidden[1].cuda(1))
model.hidden=(model.hidden[0].cuda(1),model.hidden[1].cuda(1))


In [92]:
def set_gpu_recursive(var, gpu_id):
    for key in var:
        if isinstance(var[key], dict):
            var[key] = set_gpu_recursive(var[key], gpu_id)
        else:
            try:
                var[key] = var[key].cuda(gpu_id)
            except:
                pass
    return var



In [100]:
optimizer.state = set_gpu_recursive(optimizer.state, 1)
learner.optimizer.state = set_gpu_recursive(learner.optimizer.state, 1)

In [103]:
learner.run_epochs(train_tokens,valid_tokens,1)

Batch:500 4.215455349445343  0.2790599019825459
Batch:1000 4.210862180948258  0.2788794076293707
Batch:1500 4.212145571072896  0.2783448839088281
Batch:2000 4.211311246871948  0.278226111009717
Batch:2500 4.210199825286865  0.27802792423963546
Batch:3000 4.209871447563171  0.27792235642671587
Batch:3500 4.2090058584213255  0.27796406232459203
Batch:4000 4.207844992935658  0.2780102457590401
Batch:4500 4.207080701986949  0.27803181937999194
Batch:5000 4.206981196069718  0.2780740782529116
Batch:5500 4.205610530679876  0.27823452775315805
Epoch:0 Loss:4.20557110817913
Batch:500 3.911064175128937  0.30575276267528534
Epoch:0 Loss:4.20557110817913 Accuracy:0.27822952477167406 Loss:3.910363855164207 Accuracy:0.30540491747513915


In [104]:
np.exp(3.9), np.exp(3.91)

(49.40244910553017, 49.89895197340787)

In [106]:
torch.save(model.state_dict(),f'{PATH}/inter/model_awd_lstm_state_dict_cuda_1')
torch.save(optimizer.state_dict(),f'{PATH}/inter/learner_awd_lstm_state_dict_cuda_1')


In [107]:
torch.save (model,f'{PATH}/inter/model_awd_lstm_cuda_1')


In [108]:
torch.save (optimizer,f'{PATH}/inter/optimizer_awd_lstm_cuda_1')


In [109]:
torch.save (learner,f'{PATH}/inter/learner_awd_lstm_cuda_1')

## Transfer Learning to predict RNN